In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/Users/vaishak/Desktop/Predictive Maintenance for Aircraft Engines/airbus-predictive-maintenance/data")
RAW = DATA_DIR / "raw"
PROC = DATA_DIR / "processed"
PROC.mkdir(parents=True, exist_ok=True)

cols = (
    ['unit_number','time_in_cycles','op_setting_1','op_setting_2','op_setting_3']
    + [f"sensor_{i:02d}" for i in range(1,22)]
)

train = pd.read_csv(RAW/"train_FD001.txt", sep=r"\s+", header=None, names=cols, engine="python")
test  = pd.read_csv(RAW/"test_FD001.txt",  sep=r"\s+", header=None, names=cols, engine="python")
rul   = pd.read_csv(RAW/"RUL_FD001.txt",   sep=r"\s+", header=None, names=["RUL"], usecols=[0], engine="python")

train.shape, test.shape, rul.shape


((20631, 26), (13096, 26), (100, 1))

RUL = max_cycle_per_unit − current_cycle (optional cap to limit huge targets).

In [2]:
# Compute per-unit max cycle
max_cycle = train.groupby("unit_number")["time_in_cycles"].max().rename("max_cycle")
train = train.merge(max_cycle, on="unit_number", how="left")
train["RUL"] = (train["max_cycle"] - train["time_in_cycles"]).astype(int)

# Optional: cap RUL (common practice in C-MAPSS), e.g., 125
MAX_RUL = 125
train["RUL_capped"] = train["RUL"].clip(upper=MAX_RUL)

train.head()


,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,...,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,max_cycle,RUL,RUL_capped
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,192,191,125
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,192,190,125
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,192,189,125
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,192,188,125
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,192,187,125


For test, NASA provides RUL at the last cycle of each unit. We’ll compute per-unit last cycle rows, attach provided RUL, and then propagate backward within each unit.

In [3]:
# Last cycle per test engine
last_cycle = test.groupby("unit_number")["time_in_cycles"].max().rename("last_cycle")
test = test.merge(last_cycle, on="unit_number", how="left")

# Map provided RUL rows to units in ascending order
# NASA ordering: rul rows correspond to units sorted by unit_number
rul_ordered = rul.copy().reset_index(drop=True)
units_sorted = sorted(test["unit_number"].unique())
assert len(units_sorted) == len(rul_ordered)

unit_to_terminal_rul = {u: int(rul_ordered.iloc[i,0]) for i,u in enumerate(units_sorted)}

# For each row: RUL = terminal_rul + (last_cycle - current_cycle)
test["terminal_rul"] = test["unit_number"].map(unit_to_terminal_rul)
test["RUL"] = (test["terminal_rul"] + (test["last_cycle"] - test["time_in_cycles"])).astype(int)
test["RUL_capped"] = test["RUL"].clip(upper=MAX_RUL)

test.head()


,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,last_cycle,terminal_rul,RUL,RUL_capped
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,0.03,392,2388,100.0,38.86,23.3735,31,112,142,125
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,0.03,393,2388,100.0,39.02,23.3916,31,112,141,125
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,0.03,393,2388,100.0,39.08,23.4166,31,112,140,125
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,0.03,391,2388,100.0,39.00,23.3737,31,112,139,125
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,0.03,390,2388,100.0,38.99,23.4130,31,112,138,125


Choose sensors & create rolling statistics

C-MAPSS has 21 sensors; not all are informative. A common informative subset for FD001:
sensor_02, sensor_03, sensor_04, sensor_07, sensor_08, sensor_09, sensor_11, sensor_12, sensor_13, sensor_14, sensor_15, sensor_17, sensor_20, sensor_21

In [4]:
sensor_cols = [f"sensor_{i:02d}" for i in [2,3,4,7,8,9,11,12,13,14,15,17,20,21]]
op_cols = ["op_setting_1","op_setting_2","op_setting_3"]

def add_rolling_features(df, window=20):
    df = df.sort_values(["unit_number","time_in_cycles"]).copy()
    for c in sensor_cols:
        df[f"{c}_mean_w{window}"] = df.groupby("unit_number")[c].rolling(window, min_periods=5).mean().reset_index(level=0, drop=True)
        df[f"{c}_std_w{window}"]  = df.groupby("unit_number")[c].rolling(window, min_periods=5).std().reset_index(level=0, drop=True)
        # simple slope/trend: difference between current and value 'window' steps ago divided by window
        df[f"{c}_trend_w{window}"] = (
            df.groupby("unit_number")[c].diff(window) / window
        )
    return df

train_fe = add_rolling_features(train, window=20)
test_fe  = add_rolling_features(test,  window=20)

# Fill early NaNs from rolling windows
train_fe = train_fe.fillna(method="bfill")
test_fe  = test_fe.fillna(method="bfill")


/var/folders/56/clh4qrjn7j94gs3b9wkrmlk40000gn/T/ipykernel_1648/963144155.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_fe = train_fe.fillna(method="bfill")
/var/folders/56/clh4qrjn7j94gs3b9wkrmlk40000gn/T/ipykernel_1648/963144155.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_fe  = test_fe.fillna(method="bfill")


feature set for classical ML

In [5]:
feature_cols = op_cols + sensor_cols \
    + [f"{c}_mean_w20" for c in sensor_cols] \
    + [f"{c}_std_w20" for c in sensor_cols] \
    + [f"{c}_trend_w20" for c in sensor_cols]

target_col = "RUL_capped"

X_train_full = train_fe[feature_cols]
y_train_full = train_fe[target_col]

X_test_full  = test_fe[feature_cols]
y_test_full  = test_fe[target_col]

X_train_full.shape, X_test_full.shape


((20631, 59), (13096, 59))

Scale & split (train/validation) for ML

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Split by rows (fast baseline). For purists, you can split by unit_number later.
X_tr, X_val, y_tr, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_tr_s  = scaler.fit_transform(X_tr)
X_val_s = scaler.transform(X_val)
X_test_s= scaler.transform(X_test_full)

# Save processed CSVs and scaler
pd.DataFrame(X_tr, columns=feature_cols).to_csv(PROC/"X_train.csv", index=False)
pd.DataFrame(X_val, columns=feature_cols).to_csv(PROC/"X_valid.csv", index=False)
pd.DataFrame(X_test_full, columns=feature_cols).to_csv(PROC/"X_test.csv", index=False)

y_tr.to_csv(PROC/"y_train.csv", index=False)
y_val.to_csv(PROC/"y_valid.csv", index=False)
y_test_full.to_csv(PROC/"y_test.csv", index=False)

joblib.dump(scaler, PROC/"scaler.joblib")
print("Saved processed datasets and scaler.")


Saved processed datasets and scaler.


Prepare windowed sequences for LSTM 

We’ll prepare sequences of length W per engine for time-series modeling.

In [7]:
W = 30  # sequence length

def make_sequences(df, feature_cols, target_col, window=W):
    X_seqs, y_vals = [], []
    for u, g in df.sort_values(["unit_number","time_in_cycles"]).groupby("unit_number"):
        arr = g[feature_cols].values
        tgt = g[target_col].values
        if len(arr) < window: 
            continue
        for i in range(window, len(arr)+1):
            X_seqs.append(arr[i-window:i])
            y_vals.append(tgt[i-1])
    return np.array(X_seqs), np.array(y_vals)

X_seq_train, y_seq_train = make_sequences(train_fe, feature_cols, target_col, window=W)
X_seq_test,  y_seq_test  = make_sequences(test_fe,  feature_cols, target_col, window=W)

# Scale sequences with the same scaler (fit on train only)
# reshape to 2D -> scale -> back to 3D
def scale_sequences(X3d, scaler):
    n, t, f = X3d.shape
    X2d = X3d.reshape(n*t, f)
    X2d_s = scaler.transform(X2d)
    return X2d_s.reshape(n, t, f)

# re-fit scaler on full train features (2D) for sequence use
from sklearn.preprocessing import StandardScaler
seq_scaler = StandardScaler().fit(train_fe[feature_cols])

X_seq_train_s = scale_sequences(X_seq_train, seq_scaler)
X_seq_test_s  = scale_sequences(X_seq_test,  seq_scaler)

np.save(PROC/"X_seq_train.npy", X_seq_train_s)
np.save(PROC/"y_seq_train.npy", y_seq_train)
np.save(PROC/"X_seq_test.npy",  X_seq_test_s)
np.save(PROC/"y_seq_test.npy",  y_seq_test)

import joblib
joblib.dump(seq_scaler, PROC/"seq_scaler.joblib")
X_seq_train_s.shape, X_seq_test_s.shape


/Users/vaishak/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/vaishak/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


((17731, 30, 59), (10196, 30, 59))

Run a fast Random Forest to confirm features are sane.

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_tr_s, y_tr)
pred_val = rf.predict(X_val_s)
print("Baseline RF MAE (val):", mean_absolute_error(y_val, pred_val))


Baseline RF MAE (val): 7.068142718681851
